# Module 3: Dimensionality reduction - Exercise

In this session you will compare both **feature selection** and **feature extraction** for dimensionality reduction, 
but we will still use **Naive Bayes** model.
We will apply these techniques to a real-world [HR Analytics dataset](https://www.kaggle.com/giripujar/hr-analytics).


The overall goal is to predict if an employee would leave the company.

##### NOTE

The goal is the get the best performance from the **Naive Bayes** model using feature reduction techniques. 
We can expect that a different model may be more suitable, 
however we want to only have a single moving part at a time so we will stay with Naive Bayes this week.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import numpy as np
import pandas as pd
np.random.seed(18937)

# Add code below this comment
# to import additional libraries
# and classes (Question #E101)
# ---------------------------




# Load dataset

Load dataset into a DataFrame and shuffle.

In [ ]:
# Dataset location
DATASET = '/dsa/data/all_datasets/HR_analytics.csv'
assert os.path.exists(DATASET)

# Add code below this comment (Question #E102)
# ---------------------------





Print top 5 rows of the dataset.

In [ ]:
# Add code below this comment (Question #E103)
# ---------------------------





## Processing

In the dataset, `sales` and `salary` are string columns,
which need some proccessing before we can start feature selection/extraction.

Play with following code and **add comments** to explain based on your understanding.
Feel free to drop questions on disscussion board if you encounter difficulties.

In [ ]:
# Add line-by-line comments to explain following code (Question #E104)
# ===============================
encoder = LabelBinarizer()
salary_features = encoder.fit_transform(dataset.salary)
for j, _class in enumerate(encoder.classes_):
    dataset.loc[:, 'salary_{}'.format(_class)] = salary_features[:, j]

dataset.head()

In [ ]:
# help on fit_transform
help(encoder.fit_transform)

Now please write similar code for "sales" column like what we just did above. Feel free to drop questions on disscussion board if you need clarification.

In [ ]:
# Add code below this comment (Question #E105)
# ---------------------------




# ---------------------------

dataset.info()

Store feature columns (all columns except 'left', 'sales' and 'salary') to a new variable **X**. Here `left` is the class variable.

Store column(s) to be predicted into a new variable **y**.

In [ ]:
# Add code below this comment (Question #E106)
# ---------------------------






Create train/validate split (20% validation ratio) and use variable names: **X_train, X_test, y_train, y_test**.

In [ ]:
# Add code below this comment (Question #E107)
# ---------------------------







# Feature selection and evaluation

In this section you could make use of any method so far introduced or provided by sklearn for **feature selection**.

Steps:
1. Initialize a feature selector
2. Fit feature selector on training set
3. Print indices of features selected, the "support".
4. Train a Gaussian Naive Bayes model on selected features from training set.
5. Evaluate the model by measuring its accuracy on the test set.

Please **briefly answer the 2 questions** at the end of this section; use evidence printed from your code as appropiate to support your answer. 

It is not required but if you are comparing different feature selection methods,
please add new notebook cells for each method.

In [ ]:
# Add code below this comment (Question #E108)
# ---------------------------





How many number of features did you finally choose? Why?

# Feature extraction

In this section you could use **Principle Component Analysis** or **Factor Analysis** for feature extraction.

Steps:
1. Initialize either PCA or FactorAnalysis
2. Fit feature extractor on training set.
4. Train a Gaussian Naive Bayes model on selected features from training set.
5. Evaluate the model by measuring its accuracy on the test set.

In [ ]:
# Add code below this comment (Question #E110)
# ---------------------------





Create a scree plot of **explained variance ratio** for extracted features.

In [ ]:
# Add code below this comment (Question #E111)
# ---------------------------





Do you think this feature extraction method performed satisfactorily?
How does it compare to feature selection methods in terms of effectiveness?

How would you explain the features you have extracted to somebody without a data science background if you were to recommend this method?
(short answer)

# Save your notebook!  Then `File > Close and Halt`